# Project: 멋진 챗봇 만들기

지난 노드에서 **챗봇과 번역기는 같은 집안**이라고 했던 말을 기억하시나요?    
앞서 배운 Seq2seq번역기와 Transfomer번역기에 적용할 수도 있겠지만, 이번 노드에서 배운 번역기 성능 측정법을 챗봇에도 적용해봅시다. 배운 지식을 다양하게 활용할 수 있는 것도 중요한 능력이겠죠. 이번 프로젝트를 통해서 챗봇과 번역기가 같은 집안인지 확인해보세요!

### **Step 1. 데이터 다운로드**

---

아래 링크에서 **`ChatbotData.csv`** 를 다운로드해 챗봇 훈련 데이터를 확보합니다. **`csv`** 파일을 읽는 데에는 **`pandas`** 라이브러리가 적합합니다. 읽어 온 데이터의 질문과 답변을 각각 **`questions`**, **`answers`** 변수에 나눠서 저장하세요!

- [songys/Chatbot_data](https://github.com/songys/Chatbot_data)

**☁️ 클라우드 이용자**는 심볼릭 링크를 생성하시면, 데이터를 다운로드를 할 필요가 없습니다.

```
$wget https://github.com/songys/Chatbot_data/blob/master/ChatbotData%20.csv

$mv ChatbotData\ .csv ~/aiffel/transformer_chatbot


```

In [248]:
import numpy as np
import pandas as pd
import tensorflow as tf

import re
import os
import io
import time
import random

from sklearn.model_selection import train_test_split

print(tf.__version__)

2.4.1


In [249]:
path_to_file = os.getenv('HOME') + '/aiffel/transformer_chatbot/ChatbotData .csv'
data = pd.read_csv(path_to_file)
data

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0
...,...,...,...
11818,훔쳐보는 것도 눈치 보임.,티가 나니까 눈치가 보이는 거죠!,2
11819,훔쳐보는 것도 눈치 보임.,훔쳐보는 거 티나나봐요.,2
11820,흑기사 해주는 짝남.,설렜겠어요.,2
11821,힘든 연애 좋은 연애라는게 무슨 차이일까?,잘 헤어질 수 있는 사이 여부인 거 같아요.,2


In [269]:
# Q데이터 A데이터 저장하기
src = []
tgt = []
for s,t in zip(data['Q'],data['A']):
    src.append(str(s))
    tgt.append(str(t))

In [270]:
src[0]

'12시 땡!'

In [271]:
len(src)

11823

In [272]:
tgt[0]

'하루가 또 가네요.'

In [273]:
len(tgt)

11823

### **Step 2. 데이터 정제**

---

아래 조건을 만족하는 **`preprocess_sentence()`** 함수를 구현하세요.

1. 영문자의 경우, **모두 소문자로 변환**합니다.
2. 영문자와 한글, 숫자, 그리고 주요 특수문자를 제외하곤 **정규식을 활용하여 모두 제거**합니다.

*문장부호 양옆에 공백을 추가하는 등 이전과 다르게 생략된 기능들은 우리가 사용할 토크나이저가 지원하기 때문에 굳이 구현하지 않아도 괜찮습니다!*

In [274]:
from konlpy.tag import Mecab
mecab = Mecab()

def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()

    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = re.sub(r'[" "]+', " ", sentence)
    sentence = re.sub(r"[^0-9ㄱ-ㅎㅏ-ㅣ가-힣a-zA-Z?.!,]+", " ", sentence)

    sentence = sentence.strip()
    corpus = mecab.morphs(sentence)
    
    return corpus

In [275]:
print(preprocess_sentence('소스 문장 데이터와 타겟 문장 데이터를 입력으로 받습니다.'))

['소스', '문장', '데이터', '와', '타', '겟', '문장', '데이터', '를', '입력', '으로', '받', '습니다', '.']


In [276]:
print(preprocess_sentence(src[0]))

['12', '시', '땡', '!']


In [277]:
print(preprocess_sentence(tgt[0]))

['하루', '가', '또', '가', '네요', '.']


### **Step 3. 데이터 토큰화**

---

토큰화에는 *KoNLPy*의 **`mecab`** 클래스를 사용합니다.

아래 조건을 만족하는 **`build_corpus()`** 함수를 구현하세요!

1. **소스 문장 데이터**와 **타겟 문장 데이터**를 입력으로 받습니다.
2. 데이터를 앞서 정의한 **`preprocess_sentence()`** 함수로 **정제하고, 토큰화**합니다.
3. 토큰화는 **전달받은 토크나이즈 함수를 사용**합니다. 이번엔 **`mecab.morphs`** 함수를 전달하시면 됩니다.
4. 토큰의 개수가 일정 길이 이상인 문장은 **데이터에서 제외**합니다.
5. **중복되는 문장은 데이터에서 제외**합니다. **`소스 : 타겟`** 쌍을 비교하지 않고 소스는 소스대로 타겟은 타겟대로 검사합니다. 중복 쌍이 흐트러지지 않도록 유의하세요!

구현한 함수를 활용하여 **`questions`** 와 **`answers`** 를 각각 **`que_corpus`** , **`ans_corpus`** 에 토큰화하여 저장합니다.

In [278]:
cleaned_corpus = list(set(zip(src,tgt)))

In [279]:
src_corpus = []
tgt_corpus = []


for tmp in cleaned_corpus:
    #print(tmp[0])
    #print(tmp[1])
    tmp_src = preprocess_sentence(tmp[0])
    tmp_tgt = preprocess_sentence(tmp[1])
    #if len(tmp_ko) <= 40:
    src_corpus.append(tmp_src)
    tgt_corpus.append(tmp_tgt)

    

print(len(src_corpus))
print(len(tgt_corpus))
print("Questions", src_corpus[100])   
print("Answers:", tgt_corpus[100])

11750
11750
Questions ['지금', '사귀', '고', '있', '는', '사람', '이랑', '결혼', '하', '고', '싶', '어']
Answers: ['같이', '살', '자고', '프로', '포즈', '해', '보', '세요', '.']


In [280]:
src_corpus[:10]

[['보', '고', '프', '다'],
 ['좋',
  '아',
  '하',
  '는',
  '사람',
  '이',
  '단',
  '톡',
  '에서',
  '다른',
  '남자',
  '한테',
  '관심',
  '있',
  '는',
  '것',
  '같이',
  '보여요',
  '.'],
 ['내게', '전화', '라도', '해', '줬', '으면', '좋', '겠', '다', '.'],
 ['어이없', '어'],
 ['전공', '수업', '노', '잼'],
 ['이', '발', '어떻게', '할까'],
 ['지금', '이', '겨울', '이', '라', '참', '다행', '이', '이', '네'],
 ['이젠', '놓', '아', '줘야', '할', '때', '인가', '보', '다'],
 ['성형', '할까'],
 ['스트레스', '팍팍']]

In [281]:
tgt_corpus[:10]

[['그럴', '시기', '에', '요', '.'],
 ['그렇게', '느낀다면', '조금', '씩', '정리', '하', '는', '게', '좋', '겠', '어요', '.'],
 ['기다리', '지', '마세요', '.'],
 ['그냥', '잊어버리', '세요', '.'],
 ['다른', '곳', '에', '관심', '이', '많', '은가', '봐요', '.'],
 ['짧', '게', '변화', '를', '줘도', '괜찮', '을', '거', '같', '아요', '.'],
 ['제', '가', '곁', '에', '있', '을게요', '.'],
 ['이별', '의', '끝', '을', '인정', '하', '는', '것', '도', '용기', '입니다', '.'],
 ['돈', '많이', '들', '텐데요', '.'],
 ['소리', '를', '질러', '보', '세요', '.']]

In [282]:
que_corpus = src_corpus
ans_corpus = tgt_corpus

### **Step 4. Augmentation**

---

우리에게 주어진 데이터는 **1만 개가량으로 적은 편**에 속합니다. 이럴 때에 사용할 수 있는 테크닉을 배웠으니 활용해봐야겠죠? **Lexical Substitution을 실제로 적용**해보도록 하겠습니다.

아래 링크를 참고하여 **한국어로 사전 훈련된 Embedding 모델을 다운로드**합니다. **`Korean (w)`** 가 Word2Vec으로 학습한 모델이며 용량도 적당하므로 사이트에서 **`Korean (w)`**를 찾아 다운로드하고, **`ko.bin`** 파일을 얻으세요!

- [Kyubyong/wordvectors](https://github.com/Kyubyong/wordvectors)

다운로드한 모델을 활용해 **데이터를 Augmentation** 하세요! 앞서 정의한 **`lexical_sub()`** 함수를 참고하면 도움이 많이 될 겁니다.

*Augmentation된 **`que_corpus`** 와 원본 **`ans_corpus`** 가 병렬을 이루도록, 이후엔 반대로 원본 **`que_corpus`** 와 Augmentation된 **`ans_corpus`** 가 병렬을 이루도록 하여 **전체 데이터가 원래의 3배가량으로 늘어나도록** 합니다.*

```
gnesim 버전 다운그레이드


import gensim
gensim.__version__
```

In [283]:
import gensim
print(gensim.__version__)
print(tf.__version__)

3.8.3
2.4.1


In [284]:
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

ko_dir_path = os.getenv('HOME') + '/aiffel/transformer_chatbot/ko.bin'

#loaded_model = KeyedVectors.load_word2vec_format(ko_dir_path, binary=True)
word2vec = Word2Vec.load(ko_dir_path)

print("=슝")

=슝


In [285]:
word2vec.similar_by_word("짜증")

/home/aiffel-dj60/anaconda3/envs/aiffel/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `similar_by_word` (Method will be removed in 4.0.0, use self.wv.similar_by_word() instead).
  """Entry point for launching an IPython kernel.


[('싫증', 0.7431163787841797),
 ('흠집', 0.5909335613250732),
 ('흉내', 0.5880371928215027),
 ('아물', 0.5121865272521973),
 ('울음소리', 0.5063954591751099),
 ('탄로', 0.4944569766521454),
 ('그러', 0.4920297861099243),
 ('내보', 0.48765820264816284),
 ('악취', 0.48633819818496704),
 ('이거', 0.48398828506469727)]

In [286]:
word2vec.similar_by_word("분노")

/home/aiffel-dj60/anaconda3/envs/aiffel/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `similar_by_word` (Method will be removed in 4.0.0, use self.wv.similar_by_word() instead).
  """Entry point for launching an IPython kernel.


[('격노', 0.7377744913101196),
 ('격분', 0.7231990694999695),
 ('원망', 0.721239447593689),
 ('분개', 0.6994378566741943),
 ('질투', 0.6872379183769226),
 ('실망', 0.6750858426094055),
 ('당황', 0.6650348901748657),
 ('증오', 0.6568769216537476),
 ('경악', 0.6351868510246277),
 ('반발', 0.629754900932312)]

In [288]:

# Lexical Substitution 구현하기
def lexical_sub(sentence, word2vec):
    import random

    res = ""
    toks = sentence

    try:
        _from = random.choice(toks)
        _to = word2vec.most_similar(_from)[0][0]

    except:   # 단어장에 없는 단어|
        return None

    for tok in toks:
        if tok is _from: res += _to + " "
        else: res += tok + " "

    return res

In [289]:
print(que_corpus[100])
print(lexical_sub(que_corpus[1], word2vec))

['지금', '사귀', '고', '있', '는', '사람', '이랑', '결혼', '하', '고', '싶', '어']
좋 아 하 는 사람 이 단 카카오 에서 다른 남자 한테 관심 있 는 것 같이 보여요 . 


/home/aiffel-dj60/anaconda3/envs/aiffel/lib/python3.7/site-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  # Remove the CWD from sys.path while we load stuff.


In [290]:
from tqdm import tqdm_notebook

new_que_corpus = []
new_ans_corpus = []

# Augmentation된 que_corpus 와 원본 ans_corpus 가 병렬을 이루도록
for idx in tqdm_notebook(range(len(que_corpus))):
    que_augmented = lexical_sub(que_corpus[idx], word2vec)
    ans = ans_corpus[idx]
    
    if que_augmented is not None:
        new_que_corpus.append(que_augmented.split())
        new_ans_corpus.append(ans)
        
    else:
       
        continue
    
for idx in tqdm_notebook(range(len(ans_corpus))):
    que = que_corpus[idx]
    ans_augmented = lexical_sub(ans_corpus[idx], word2vec)
    
    if ans_augmented is not None:
        new_que_corpus.append(que)
        new_ans_corpus.append(ans_augmented.split())
       
    else:
       
        continue

/home/aiffel-dj60/anaconda3/envs/aiffel/lib/python3.7/site-packages/ipykernel_launcher.py:7: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  import sys


  0%|          | 0/11750 [00:00<?, ?it/s]

/home/aiffel-dj60/anaconda3/envs/aiffel/lib/python3.7/site-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  # Remove the CWD from sys.path while we load stuff.
/home/aiffel-dj60/anaconda3/envs/aiffel/lib/python3.7/site-packages/ipykernel_launcher.py:19: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/11750 [00:00<?, ?it/s]

In [291]:
print(new_que_corpus[:10])
print(new_ans_corpus[:10])

[['살펴보', '고', '프', '다'], ['좋', '아', '하', '는', '사람', '이', '단', '톡', '오세아니아', '다른', '남자', '한테', '관심', '있', '는', '것', '같이', '보여요', '.'], ['독학', '수업', '노', '잼'], ['그러', '발', '어떻게', '할까'], ['지금', '이', '겨울', '이', '라', '참', '다행', '이', '그러', '네'], ['이젠', '놓', '아서', '줘야', '할', '때', '인가', '보', '다'], ['한강', '오세아니아', '소주', '한', '잔', '.'], ['짝사랑', '했', '던', '그', '한테', '여자', '친구', '가', '생겼', '어', '.'], ['소개팅', '거절', '때문', '힘든', '거', '네', '.'], ['연락', '죽', '더라도', '안', '올', '텐데', '매일', '기다려']]
[['그럴', '시기', '에', '요', '.'], ['그렇게', '느낀다면', '조금', '씩', '정리', '하', '는', '게', '좋', '겠', '어요', '.'], ['다른', '곳', '에', '관심', '이', '많', '은가', '봐요', '.'], ['짧', '게', '변화', '를', '줘도', '괜찮', '을', '거', '같', '아요', '.'], ['제', '가', '곁', '에', '있', '을게요', '.'], ['이별', '의', '끝', '을', '인정', '하', '는', '것', '도', '용기', '입니다', '.'], ['분위기', '있', '네요', '.'], ['마음', '이', '아프', '겠', '네요', '.'], ['안', '힘든', '게', '없', '네요', '.'], ['변해야할', '시기', '를', '놓친', '걸', '수', '도', '있', '어요', '.']]


In [292]:
print(len(new_que_corpus))
print(len(new_ans_corpus))

20444
20444


### **Step 5. 데이터 벡터화**

---

타겟 데이터인 **`ans_corpus`** 에 **`<start>`** 토큰과 **`<end>`** 토큰이 추가되지 않은 상태이니 이를 먼저 해결한 후 벡터화를 진행합니다. 우리가 구축한 **`ans_corpus`** 는 **`list`** 형태이기 때문에 아주 쉽게 이를 해결할 수 있답니다!

`sample_data = ["12", "시", "땡", "!"]`

``

`print(["<start>"] + sample_data + ["<end>"])`

1. 위 소스를 참고하여 타겟 데이터 전체에 **`<start>`** 토큰과 **`<end>`** 토큰을 추가해 주세요!

챗봇 훈련 데이터의 가장 큰 특징 중 하나라고 하자면 바로 **소스 데이터와 타겟 데이터가 같은 언어를 사용한다는 것**이겠죠. 앞서 배운 것처럼 이는 Embedding 층을 공유했을 때 많은 이점을 얻을 수 있습니다.

1. 특수 토큰을 더함으로써 **`ans_corpus`** 또한 완성이 되었으니, **`que_corpus`** 와 결합하여 **전체 데이터에 대한 단어 사전을 구축**하고 **벡터화하여 `enc_train` 과 `dec_train`** 을 얻으세요!

In [293]:
tgt_corpus = []

for corpus in ans_corpus:
    tgt_corpus.append(["<start>"] + corpus + ["<end>"])
    
print(tgt_corpus[0])
print(tgt_corpus[325])
print(tgt_corpus[395])
ans_corpus = tgt_corpus

['<start>', '그럴', '시기', '에', '요', '.', '<end>']
['<start>', '떨리', '는', '감정', '은', '그', '자체', '로', '소중', '해요', '.', '<end>']
['<start>', '후회', '는', '후회', '를', '낳', '을', '뿐', '이', '에요', '.', '용기', '내', '세요', '.', '<end>']


In [294]:
from collections import Counter

voc_data = que_corpus + ans_corpus

words = np.concatenate(voc_data).tolist()
counter = Counter(words)
counter = counter.most_common(30000-2)
vocab = ['<pad>', '<unk>'] + [key for key, _ in counter]
word_to_index = {word:index for index, word in enumerate(vocab)}
index_to_word = {index:word for word, index in word_to_index.items()}

In [316]:
#index_to_word

In [295]:
def get_encoded_sentence(sentence, word_to_index):
    return [word_to_index[word] if word in word_to_index else word_to_index['<unk>'] for word in sentence]

def get_decoded_sentence(encoded_sentence, index_to_word):
    return ' '.join(index_to_word[index] if index in index_to_word else '<unk>' for index in encoded_sentence[1:])  #[1:]를 통해 <BOS>를 제외

def vectorize(corpus, word_to_index):
    data = []
    for sen in corpus:
        sen = get_encoded_sentence(sen, word_to_index)
        data.append(sen)
    return data

que_train = vectorize(que_corpus, word_to_index)
ans_train = vectorize(ans_corpus, word_to_index)

print(len(que_train))
print(len(ans_train))


11750
11750


In [296]:
enc_tensor = tf.keras.preprocessing.sequence.pad_sequences(que_train, padding='post')
dec_tensor = tf.keras.preprocessing.sequence.pad_sequences(ans_train, padding='post')

enc_train, enc_val, dec_train, dec_val = \
train_test_split(enc_tensor, dec_tensor, test_size=0.01) # test set은 1%만

print(len(enc_train))
print(len(enc_val)) 
print(len(dec_train))
print(len(dec_val))

11632
118
11632
118


In [317]:
enc_train[0]

array([  69,   43,    9,  305, 4822,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0],
      dtype=int32)

In [318]:
dec_train[0]

array([   3,  575,    5, 4285,  186, 1118,    9, 4285,  186,  648,   10,
          2,    4,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0], dtype=int32)

In [299]:
print(len(enc_train[0]))
print(len(dec_train[0]))

32
42


# 모델링

In [300]:
# Positional Encoding 구현
def positional_encoding(pos, d_model):
    def cal_angle(position, i):
        return position / np.power(10000, int(i) / d_model)

    def get_posi_angle_vec(position):
        return [cal_angle(position, i) for i in range(d_model)]

    sinusoid_table = np.array([get_posi_angle_vec(pos_i) for pos_i in range(pos)])

    sinusoid_table[:, 0::2] = np.sin(sinusoid_table[:, 0::2])  # 인덱스가 짝수
    sinusoid_table[:, 1::2] = np.cos(sinusoid_table[:, 1::2])  # 인덱스가 홀수

             
    return sinusoid_table

슝=3


In [301]:
# Mask  생성하기
def generate_padding_mask(seq):
    # tf.math.equal: seq의 원소가 0이 되면 true로 반환, 아니면 false 반환
    # tf.cast: true를 float32로 변환
    
    seq = tf.cast(tf.math.equal(seq, 0), tf.float32)
    return seq[:, tf.newaxis, tf.newaxis, :]    # np.newaxis: numpy array의 차원 늘려주기

def generate_causality_mask(src_len, tgt_len):
    # np.cumsum(): 배열에서 행에 따라 누적되는 원소들의 누적합 계산
    # np.eye(): 대각선이 1인 seq_len x seq_len 크기의 대각행렬 생성
    
    mask = 1 - np.cumsum(np.eye(src_len, tgt_len), 0)
    return tf.cast(mask, tf.float32)  # tf.cast: mask(텐서)를 float32로 변환


def generate_masks(src, tgt):
    enc_mask = generate_padding_mask(src)
    dec_mask = generate_padding_mask(tgt)

    dec_causality_mask = generate_causality_mask(tgt.shape[1], tgt.shape[1])
    dec_mask = tf.maximum(dec_mask, dec_causality_mask)

    dec_enc_causality_mask = generate_causality_mask(tgt.shape[1], src.shape[1])
    dec_enc_mask = tf.maximum(enc_mask, dec_enc_causality_mask)

    return enc_mask, dec_enc_mask, dec_mask

슝=3


In [319]:
# Multi Head Attention 구현
class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads
        self.d_model = d_model

        self.depth = d_model // self.num_heads
        
        # Linear Layer
        self.W_q = tf.keras.layers.Dense(d_model)
        self.W_k = tf.keras.layers.Dense(d_model)
        self.W_v = tf.keras.layers.Dense(d_model)

        self.linear = tf.keras.layers.Dense(d_model)

    def scaled_dot_product_attention(self, Q, K, V, mask):
        d_k = tf.cast(K.shape[-1], tf.float32)
        QK = tf.matmul(Q, K, transpose_b=True)

        scaled_qk = QK / tf.math.sqrt(d_k)   #  Scaled QK

        if mask is not None: scaled_qk += (mask * -1e9)  

        attentions = tf.nn.softmax(scaled_qk, axis=-1)   # Attention Weights
        out = tf.matmul(attentions, V)

        return out, attentions


    def split_heads(self, x):
        bsz = x.shape[0]  # batch size
        # reshape - shape의 한 원소만 -1, 의미는 전체 크기가 일정하게 유지되도록 해당 차원의 길이가 자동으로 계산
        split_x = tf.reshape(x, (bsz, -1, self.num_heads, self.depth))
        split_x = tf.transpose(split_x, perm=[0, 2, 1, 3])   #  perm은 치환하는 위치를 알려줌

        return split_x

    
    def combine_heads(self, x):
        bsz = x.shape[0]
        combined_x = tf.transpose(x, perm=[0, 2, 1, 3])
        combined_x = tf.reshape(combined_x, (bsz, -1, self.d_model))

        return combined_x


    def call(self, Q, K, V, mask):
        # Linear 레이어 추가 - embedding 매핑
        WQ = self.W_q(Q)
        WK = self.W_k(K)
        WV = self.W_v(V)

        WQ_splits = self.split_heads(WQ)
        WK_splits = self.split_heads(WK)
        WV_splits = self.split_heads(WV)

        out, attention_weights = self.scaled_dot_product_attention(
            WQ_splits, WK_splits, WV_splits, mask)

        out = self.combine_heads(out)
        out = self.linear(out)

        return out, attention_weights

In [303]:
# Position-wise Feed Forward Network 구현
class PoswiseFeedForwardNet(tf.keras.layers.Layer):
    def __init__(self, d_model, d_ff):
        super(PoswiseFeedForwardNet, self).__init__()
        self.d_model = d_model
        self.d_ff = d_ff

        self.fc1 = tf.keras.layers.Dense(d_ff, activation='relu')
        self.fc2 = tf.keras.layers.Dense(d_model)

    def call(self, x):
        out = self.fc1(x)
        out = self.fc2(out)

        return out

슝=3


In [304]:
class EncoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, n_heads, d_ff, dropout):
        super(EncoderLayer, self).__init__()

        self.enc_self_attn = MultiHeadAttention(d_model, n_heads)
        self.ffn = PoswiseFeedForwardNet(d_model, d_ff)

        self.norm_1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.norm_2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.do = tf.keras.layers.Dropout(dropout)

    def call(self, x, mask):

        """
        Multi-Head Attention
        """
        residual = x
        out = self.norm_1(x)
        out, enc_attn = self.enc_self_attn(out, out, out, mask)
        out = self.do(out)
        out += residual

        """
        Position-Wise Feed Forward Network
        """
        residual = out
        out = self.norm_2(out)
        out = self.ffn(out)
        out = self.do(out)
        out += residual

        return out, enc_attn

슝=3


In [305]:
# Decoder 레이어 구현
class DecoderLayer(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(DecoderLayer, self).__init__()

        self.dec_self_attn = MultiHeadAttention(d_model, num_heads)
        self.enc_dec_attn = MultiHeadAttention(d_model, num_heads)

        self.ffn = PoswiseFeedForwardNet(d_model, d_ff)

        self.norm_1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.norm_2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.norm_3 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        self.do = tf.keras.layers.Dropout(dropout)

    def call(self, x, enc_out, causality_mask, padding_mask):

        """
        Masked Multi-Head Attention
        """
        residual = x
        out = self.norm_1(x)
        out, dec_attn = self.dec_self_attn(out, out, out, padding_mask)
        out = self.do(out)
        out += residual

        """
        Multi-Head Attention
        """
        residual = out
        out = self.norm_2(out)
        out, dec_enc_attn = self.dec_self_attn(out, enc_out, enc_out, causality_mask)
        out = self.do(out)
        out += residual

        """
        Position-Wise Feed Forward Network
        """
        residual = out
        out = self.norm_3(out)
        out = self.ffn(out)
        out = self.do(out)
        out += residual

        return out, dec_attn, dec_enc_attn
print("슝=3")

슝=3


In [306]:
# Encoder 구현
class Encoder(tf.keras.Model):
    def __init__(self,
                    n_layers,
                    d_model,
                    n_heads,
                    d_ff,
                    dropout):
        super(Encoder, self).__init__()
        self.n_layers = n_layers
        self.enc_layers = [EncoderLayer(d_model, n_heads, d_ff, dropout) 
                        for _ in range(n_layers)]

        self.do = tf.keras.layers.Dropout(dropout)

    def call(self, x, mask):
        out = x

        enc_attns = list()
        for i in range(self.n_layers):
            out, enc_attn = self.enc_layers[i](out, mask)
            enc_attns.append(enc_attn)

        return out, enc_attns
print("슝=3")


슝=3


In [307]:
# Decoder 구현
class Decoder(tf.keras.Model):
    def __init__(self,
                    n_layers,
                    d_model,
                    n_heads,
                    d_ff,
                    dropout):
        super(Decoder, self).__init__()
        self.n_layers = n_layers
        self.dec_layers = [DecoderLayer(d_model, n_heads, d_ff, dropout) 
                            for _ in range(n_layers)]


    def call(self, x, enc_out, causality_mask, padding_mask):
        out = x

        dec_attns = list()
        dec_enc_attns = list()
        for i in range(self.n_layers):
            out, dec_attn, dec_enc_attn = \
            self.dec_layers[i](out, enc_out, causality_mask, padding_mask)

            dec_attns.append(dec_attn)
            dec_enc_attns.append(dec_enc_attn)

        return out, dec_attns, dec_enc_attns

슝=3


In [320]:
class Transformer(tf.keras.Model):
    def __init__(self,
                    n_layers,
                    d_model,
                    n_heads,
                    d_ff,  # 레이어의 차원수
                    src_vocab_size,
                    tgt_vocab_size,
                    pos_len,
                    dropout=0.2,
                    shared_fc=True,
                    shared_emb=False):
        super(Transformer, self).__init__()
    
        self.d_model = tf.cast(d_model, tf.float32)

        if shared_emb:
            self.enc_emb = self.dec_emb = \
            tf.keras.layers.Embedding(src_vocab_size, d_model)
        else:
            self.enc_emb = tf.keras.layers.Embedding(src_vocab_size, d_model)
            self.dec_emb = tf.keras.layers.Embedding(tgt_vocab_size, d_model)

        self.pos_encoding = positional_encoding(pos_len, d_model)
        self.do = tf.keras.layers.Dropout(dropout)

        self.encoder = Encoder(n_layers, d_model, n_heads, d_ff, dropout)
        self.decoder = Decoder(n_layers, d_model, n_heads, d_ff, dropout)

        self.fc = tf.keras.layers.Dense(tgt_vocab_size)

        self.shared_fc = shared_fc

        if shared_fc:
            self.fc.set_weights(tf.transpose(self.dec_emb.weights))

    def embedding(self, emb, x):
  
        seq_len = x.shape[1]

        out = emb(x)

        if self.shared_fc: out *= tf.math.sqrt(self.d_model)

        out += self.pos_encoding[np.newaxis, ...][:, :seq_len, :]
        # np.newaxis: numpy array의 차원 늘려주기
        
        out = self.do(out)

        return out


    def call(self, enc_in, dec_in, enc_mask, causality_mask, dec_mask):
        
        enc_in = self.embedding(self.enc_emb, enc_in)
        dec_in = self.embedding(self.dec_emb, dec_in)

        enc_out, enc_attns = self.encoder(enc_in, enc_mask)

        dec_out, dec_attns, dec_enc_attns = \
        self.decoder(dec_in, enc_out, causality_mask, dec_mask)

        logits = self.fc(dec_out)

        return logits, enc_attns, dec_attns, dec_enc_attns

In [328]:

n_layers = 1
d_model = 368
n_heads = 8
d_ff = 1024
dropout = 0.2

In [329]:
# 주어진 하이퍼파라미터로 Transformer 인스턴스 생성
VOCAB_SIZE = 20000

transformer = Transformer(
    n_layers= n_layers,
    d_model = d_model,
    n_heads=n_heads,
    d_ff=d_ff,
    src_vocab_size=VOCAB_SIZE,
    tgt_vocab_size=VOCAB_SIZE,
    pos_len=200,
    dropout=dropout,
    shared_fc=True,
    shared_emb=True)

In [331]:
class LearningRateScheduler(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=4000):
        super(LearningRateScheduler, self).__init__()

        self.d_model = d_model
        self.warmup_steps = warmup_steps

    def __call__(self, step):
        arg1 = step ** -0.5
        arg2 = step * (self.warmup_steps ** -1.5)

        return (self.d_model ** -0.5) * tf.math.minimum(arg1, arg2)

In [332]:
# Learning Rate 인스턴스 선언 & Optimizer 구현
learning_rate = LearningRateScheduler(d_model)

optimizer = tf.keras.optimizers.Adam(learning_rate,
                                        beta_1=0.9,
                                        beta_2=0.98, 
                                        epsilon=1e-9)

In [333]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True, reduction='none')

def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)

    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_sum(loss_)/tf.reduce_sum(mask)

In [334]:

# Train Step 정의
@tf.function()
def train_step(src, tgt, model, optimizer):
    tgt_in = tgt[:, :-1]  # Decoder의 input
    gold = tgt[:, 1:]     # Decoder의 output과 비교하기 위해 right shift를 통해 생성한 최종 타겟

    enc_mask, dec_enc_mask, dec_mask = generate_masks(src, tgt_in)

    with tf.GradientTape() as tape:
        predictions, enc_attns, dec_attns, dec_enc_attns = \
        model(src, tgt_in, enc_mask, dec_enc_mask, dec_mask)
        loss = loss_function(gold, predictions)

    gradients = tape.gradient(loss, model.trainable_variables)    
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    return loss, enc_attns, dec_attns, dec_enc_attns

In [335]:


def get_decoded_sentence(encoded_sentence, idx2word):
    return ' '.join(idx2word[index] if index in idx2word else '<UNK>' for index in encoded_sentence[1:]) 


def get_decoded_sentences(encoded_sentences, idx2word):
    return [get_decoded_sentence(encoded_sentence, idx2word) for encoded_sentence in encoded_sentences]

In [351]:
# translate()

def evaluate(sentence, model):
    # sentence 전처리(enc_train과 같은 모양으로)
    sentence = preprocess_sentence(sentence)
    pieces = sentence
    tokens = get_encoded_sentence(pieces, word_to_index)

    _input = tf.keras.preprocessing.sequence.pad_sequences([tokens],
                                                           maxlen=enc_train.shape[-1],
                                                           padding='post')
    
    ids = []
    
    output = tf.expand_dims([word_to_index["<start>"]], 0) 
    for i in range(dec_train.shape[-1]):
        enc_padding_mask, combined_mask, dec_padding_mask = \
        generate_masks(_input, output)

        predictions, enc_attns, dec_attns, dec_enc_attns =\
        model(_input, 
              output,
              enc_padding_mask,
              combined_mask,
              dec_padding_mask)

        predicted_id = \
        tf.argmax(tf.math.softmax(predictions, axis=-1)[0, -1]).numpy().item()
        
        # 예측 단어가 종료 토큰일 경우
        if word_to_index["<end>"] == predicted_id:
            result = get_decoded_sentence(ids, index_to_word)
            return pieces, result, enc_attns, dec_attns, dec_enc_attns
        ##word_to_index
        ids.append(predicted_id)
        output = tf.concat([output, tf.expand_dims([predicted_id], 0)], axis=-1)

    result = get_decoded_sentence(ids, index_to_word)

    return pieces, result, enc_attns, dec_attns, dec_enc_attns

def translate(sentence, model):
    pieces, result, enc_attns, dec_attns, dec_enc_attns = \
    evaluate(sentence, model)
    
    print('Input: %s' % (sentence))
    print('Predicted translation: {}'.format(result))

    return result

In [352]:
examples = [
    "지루하다, 놀러가고 싶어.",
    "오늘 일찍 일어났더니 피곤하다.",
    "간만에 여자친구랑 데이트 하기로 했어.",
    "집에 있는다는 소리야."
]

### **Step 6. 훈련하기**

---

앞서 번역 모델을 훈련하며 정의한 **`Transformer`** 를 그대로 사용하시면 됩니다! 대신 데이터의 크기가 작으니 하이퍼파라미터를 튜닝해야 과적합을 피할 수 있습니다. 모델을 훈련하고 아래 예문에 대한 답변을 생성하세요! **가장 멋진 답변**과 **모델의 하이퍼파라미터**를 제출하시면 됩니다.

```
# 예문1. 지루하다, 놀러가고 싶어.
2. 오늘 일찍 일어났더니 피곤하다.
3. 간만에 여자친구랑 데이트 하기로 했어.
4. 집에 있는다는 소리야.

---

# 제출Translations
> 1. 잠깐 쉬 어도 돼요 . <end>
> 2. 맛난 거 드세요 . <end>
> 3. 떨리 겠 죠 . <end>
> 4. 좋 아 하 면 그럴 수 있 어요 . <end>

Hyperparameters
> n_layers: 1
> d_model: 368
> n_heads: 8
> d_ff: 1024
> dropout: 0.2

Training Parameters
> Warmup Steps: 1000
> Batch Size: 64
> Epoch At: 10

```

In [341]:
from tqdm import tqdm_notebook 

BATCH_SIZE = 64
EPOCHS = 5

for epoch in range(EPOCHS):
    total_loss = 0

    idx_list = list(range(0, enc_train.shape[0], BATCH_SIZE))
    random.shuffle(idx_list)
    t = tqdm_notebook(idx_list)

    for (batch, idx) in enumerate(t):
        batch_loss, enc_attns, dec_attns, dec_enc_attns = \
        train_step(enc_train[idx:idx+BATCH_SIZE],
                    dec_train[idx:idx+BATCH_SIZE],
                    transformer,
                    optimizer)

        total_loss += batch_loss

        t.set_description_str('Epoch %2d' % (epoch + 1))
        t.set_postfix_str('Loss %.4f' % (total_loss.numpy() / (batch + 1)))

/home/aiffel-dj60/anaconda3/envs/aiffel/lib/python3.7/site-packages/ipykernel_launcher.py:11: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  # This is added back by InteractiveShellApp.init_path()


  0%|          | 0/182 [00:00<?, ?it/s]

  0%|          | 0/182 [00:00<?, ?it/s]

  0%|          | 0/182 [00:00<?, ?it/s]

  0%|          | 0/182 [00:00<?, ?it/s]

  0%|          | 0/182 [00:00<?, ?it/s]

In [353]:
print("Translations")   
for example in examples:
    translate(example, transformer)

Translations
Input: 지루하다, 놀러가고 싶어.
Predicted translation: 하 는 게 사람 들 과 함께 하 세요 .
Input: 오늘 일찍 일어났더니 피곤하다.
Predicted translation: 차리 세요 .
Input: 간만에 여자친구랑 데이트 하기로 했어.
Predicted translation: 는 떨리 겠 어요 .
Input: 집에 있는다는 소리야.
Predicted translation: 오 세요 .


### **Step 7. 성능 측정하기**

---

챗봇의 경우, 올바른 대답을 하는지가 중요한 평가지표입니다. 올바른 답변을 하는지 눈으로 확인할 수 있겠지만, 많은 데이터의 경우는 모든 결과를 확인할 수 없을 것입니다. 주어잔 질문에 적절한 답변을 하는지 확인하고, BLEU Score를 계산하는 **`calculate_bleu()`** 함수도 적용해보세요.

In [354]:
from nltk.translate.bleu_score import sentence_bleu

reference = "많 은 자연어 처리 연구자 들 이 트랜스포머 를 선호 한다".split()
candidate = "적 은 자연어 학 개발자 들 가 트랜스포머 을 선호 한다 요".split()

print("원문:", reference)
print("번역문:", candidate)
print("BLEU Score:", sentence_bleu([reference], candidate))

원문: ['많', '은', '자연어', '처리', '연구자', '들', '이', '트랜스포머', '를', '선호', '한다']
번역문: ['적', '은', '자연어', '학', '개발자', '들', '가', '트랜스포머', '을', '선호', '한다', '요']
BLEU Score: 8.190757052088229e-155


/home/aiffel-dj60/anaconda3/envs/aiffel/lib/python3.7/site-packages/nltk/translate/bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/home/aiffel-dj60/anaconda3/envs/aiffel/lib/python3.7/site-packages/nltk/translate/bleu_score.py:516: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


In [355]:
from nltk.translate.bleu_score import SmoothingFunction

def calculate_bleu(reference, candidate, weights=[0.25, 0.25, 0.25, 0.25]):
    return sentence_bleu([reference],
                         candidate,
                         weights=weights,
                         smoothing_function=SmoothingFunction().method1)  # smoothing_function 적용

print("BLEU-1:", calculate_bleu(reference, candidate, weights=[1, 0, 0, 0]))
print("BLEU-2:", calculate_bleu(reference, candidate, weights=[0, 1, 0, 0]))
print("BLEU-3:", calculate_bleu(reference, candidate, weights=[0, 0, 1, 0]))
print("BLEU-4:", calculate_bleu(reference, candidate, weights=[0, 0, 0, 1]))

print("\nBLEU-Total:", calculate_bleu(reference, candidate))

BLEU-1: 0.5
BLEU-2: 0.18181818181818182
BLEU-3: 0.010000000000000004
BLEU-4: 0.011111111111111112

BLEU-Total: 0.05637560315259291


In [362]:
def eval_bleu(src_corpus, tgt_corpus, verbose=True):
    total_score = 0.0
    sample_size = len(tgt_corpus)

    for idx in tqdm_notebook(range(sample_size)):
        src_tokens = src_corpus[idx]
        tgt_tokens = tgt_corpus[idx]
        
        src = []
        tgt = []
        
        for word in src_tokens:
            if word !=0 and word !=1 and word !=3 and word !=4:
                src.append(word)
        
        for word in tgt_tokens:
            if word != 0 and word != 3 and word !=4:
                tgt.append(word)

        src_sentence = get_decoded_sentence(src, index_to_word)
        tgt_sentence = get_decoded_sentence(tgt, index_to_word)
        
        
        reference = preprocess_sentence(tgt_sentence)
        candidate = translate(src_sentence, transformer)

        score = sentence_bleu([reference], candidate,
                              smoothing_function=SmoothingFunction().method1)
        total_score += score

        if verbose:
            print("Source Sentence: ", src_sentence)
            print("Model Prediction: ", candidate)
            print("Real: ", reference)
            print("Score: %lf\n" % score)

    print("Num of Sample:", sample_size)
    print("Total Score:", total_score / sample_size)

In [363]:
eval_bleu(enc_val[::5], dec_val[::5], verbose=True)

/home/aiffel-dj60/anaconda3/envs/aiffel/lib/python3.7/site-packages/ipykernel_launcher.py:5: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  """


  0%|          | 0/24 [00:00<?, ?it/s]

Input: 전 남친 흔적 찾 기
Predicted translation: 은 사람 은 모두 에게 찾아보 세요 .
Source Sentence:  전 남친 흔적 찾 기
Model Prediction:  은 사람 은 모두 에게 찾아보 세요 .
Real:  ['으려고', '노력', '하', '지', '마세요', '.']
Score: 0.009134

Input: 하 러 가 야지
Predicted translation: 아요 .
Source Sentence:  하 러 가 야지
Model Prediction:  아요 .
Real:  ['은', '뭐', '든', '좋', '아요', '.']
Score: 0.048730

Input: 이 이상 해
Predicted translation: 평범 면서 지극히 특별 하 죠 .
Source Sentence:  이 이상 해
Model Prediction:  평범 면서 지극히 특별 하 죠 .
Real:  ['이유', '인지', '생각', '해', '보', '세요', '.']
Score: 0.010802

Input: 인데 남편 이랑 자주 부딪혀
Predicted translation: 해요 !
Source Sentence:  인데 남편 이랑 자주 부딪혀
Model Prediction:  해요 !
Real:  ['다른', '삶', '을', '살', '다가', '하루', '아침', '에', '같이', '살', '게', '된', '거', '니까요', '.']
Score: 0.000000

Input: 싫 어
Predicted translation: 어 하 지 말 아요 .
Source Sentence:  싫 어
Model Prediction:  어 하 지 말 아요 .
Real:  ['도', '싫', '어요', '.']
Score: 0.017033

Input: 이 자꾸 나오 네
Predicted translation: 입니다 .
Source Sentence:  이 자꾸 나오 네
Model Prediction:  입니다 .
Real:  ['

루브릭

아래의 기준을 바탕으로 프로젝트를 평가합니다.

평가문항    
상세기준    

1. 챗봇 훈련데이터 전처리 과정이 체계적으로 진행되었는가?
      - 챗봇 훈련데이터를 위한 전처리와 augmentation이 적절히 수행되어 3만개 가량의 훈련데이터셋이 구축되었다.

2. transformer 모델을 활용한 챗봇 모델이 과적합을 피해 안정적으로 훈련되었는가?
      - 과적합을 피할 수 있는 하이퍼파라미터 셋이 적절히 제시되었다.

3. 챗봇이 사용자의 질문에 그럴듯한 형태로 답하는 사례가 있는가?
      - 주어진 예문을 포함하여 챗봇에 던진 질문에 적절히 답하는 사례가 제출되었다.
